In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from math import sqrt
import warnings
warnings.filterwarnings("ignore")

import env
import split_scale
import features

## Data Acquisition

In [4]:
query = '''
select *
from properties_2017
join predictions_2017 as pred
using(parcelid)
where pred.`transactiondate` like '%%-05-%%' or pred.`transactiondate` like '%%-06-%%'
'''
raw_df = pd.read_sql(query, env.get_db_url('zillow'))

In [5]:
raw_df.head()

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,logerror,transactiondate
0,11393337,2463969,NaN,NaN,NaN,3.0,3.0,NaN,4.0,3.0,...,43439.0,2016.0,22755.0,756.94,Y,14.0,6.037235e+13,117,0.086137,2017-06-08
1,11289917,2061546,1.0,NaN,NaN,2.0,3.0,NaN,6.0,2.0,...,136104.0,2016.0,27214.0,2319.90,Y,15.0,6.037901e+13,1248,-0.362001,2017-06-23
2,11705026,1834372,NaN,NaN,NaN,1.0,2.0,NaN,6.0,1.0,...,35606.0,2016.0,23624.0,543.69,None,NaN,6.037232e+13,1772,-0.146056,2017-06-30
3,14269464,1923117,NaN,NaN,NaN,3.0,4.0,NaN,NaN,3.0,...,880456.0,2016.0,445569.0,9819.72,None,NaN,6.059064e+13,2028,0.021085,2017-06-01
4,11389003,2121349,NaN,NaN,NaN,2.0,3.0,NaN,6.0,2.0,...,614000.0,2016.0,449000.0,7673.19,None,NaN,6.037703e+13,3273,-0.325393,2017-06-01


In [6]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21944 entries, 0 to 21943
Data columns (total 62 columns):
parcelid                        21944 non-null int64
id                              21944 non-null int64
airconditioningtypeid           7088 non-null float64
architecturalstyletypeid        49 non-null float64
basementsqft                    12 non-null float64
bathroomcnt                     21938 non-null float64
bedroomcnt                      21938 non-null float64
buildingclasstypeid             4 non-null float64
buildingqualitytypeid           13949 non-null float64
calculatedbathnbr               21762 non-null float64
decktypeid                      179 non-null float64
finishedfloor1squarefeet        1765 non-null float64
calculatedfinishedsquarefeet    21883 non-null float64
finishedsquarefeet12            20951 non-null float64
finishedsquarefeet13            12 non-null float64
finishedsquarefeet15            810 non-null float64
finishedsquarefeet50            17